Pull request data from GitHub

In [ ]:
# Imports
import requests
from dateutil.parser import parse as parse_date
import pandas as pd
from config import github_pat as pat

In [ ]:
# Set Up Code With Your Parameters

# owner and repo_names are used to construct the urls for the github api to access repo data
owner = '<the github org>'
team_slug = '<the github team name>'

repo_names = [
  'repo1',
  'repo2',
  '...',
  ]

# the date range you're interested in
start_date = '2023-08-01T00:00:00Z'
end_date = '2023-10-31T23:59:59Z'
start_week = parse_date(start_date).isocalendar().week
end_week = parse_date(end_date).isocalendar().week

In [ ]:
# Get team members
# -- this assumes there is less than 1 page of team members (100) --

headers = {
  'Accept': 'application/vnd.github+json',
  'Authorization': 'Bearer ' + pat,
  'X-GitHub-Api-Version': '2022-11-28'
  }

url = f'https://api.github.com/orgs/{owner}/teams/{team_slug}/members'

r = requests.get(url, headers=headers)
if (r.status_code != requests.codes.ok):
  r.raise_for_status()

gh_account_list = [ user['login'] for user in r.json()]

print(gh_account_list)

In [ ]:
# Make requests
per_page = 30
params = {
  'state': 'closed',
  'page': 1,
  'per_page': per_page,
  }

headers = {
  'Accept': 'application/vnd.github+json',
  'Authorization': 'Bearer ' + pat,
  'X-GitHub-Api-Version': '2022-11-28'}

pr_data_dict = dict()

for repo_name in repo_names:
  print('fetching '+repo_name+'...')
  page = 1
  url = 'https://api.github.com/repos/' + str(owner) + '/' + str(repo_name) + '/pulls' 
  pr_data_dict[repo_name] = []

  # Paginate. This might break given edge cases
  while(True):
    print('page '+str(page))
    params['page'] = page
    r = requests.get(url, headers=headers, params=params)
    pr_data_dict[repo_name].extend(r.json())
    if parse_date(pr_data_dict[repo_name][-1]['created_at']) >= parse_date(start_date) and len(r.json()) >= per_page:
      page += 1
      continue
    break

In [ ]:
# Select and clean data we want from each PR

# You can access any nested properties using '.' notation in the data_fields array, 
# such as user.login. This should work for an arbitrary depth of data
data_fields = ['id', 'number', 'created_at', 'merged_at', 'user.login']

def recursive_extract_nested_data(data_dict, complex_key):
  keys = complex_key.rsplit('.')
  if len(keys) == 1:
    if (keys[0] == 'created_at' or keys[0] == 'merged_at'):
      try:
        return parse_date(data_dict[keys[0]])
      except:
        return data_dict[keys[0]]
    return data_dict[keys[0]]
  return recursive_extract_nested_data(data_dict[keys[0]], '.'.join(keys[1:]))

def truncate_pr_data(pr_data_dict, data_fields):
  truncated_pr_data_list = []
  for pr_data in pr_data_dict:
    truncated_pr_data = {}
    for data_field in data_fields:
      truncated_pr_data[data_field] = recursive_extract_nested_data(pr_data, data_field)
    truncated_pr_data_list.append(truncated_pr_data)
  
  return truncated_pr_data_list

cleaned_pr_data = {}
for repo_name in repo_names:
  cleaned_pr_data[repo_name] = truncate_pr_data(pr_data_dict[repo_name], data_fields)

In [ ]:
# Create Dataframes in time range

data_frames = {}

for repo_name in repo_names:
  data_frames[repo_name] = pd.DataFrame(cleaned_pr_data[repo_name])
  data_frames[repo_name] = data_frames[repo_name].loc[(data_frames[repo_name].created_at >= start_date) & (data_frames[repo_name].created_at < end_date)]

data_frames[repo_names[0]].head()

In [ ]:
# Extract weekly PR data

weeks_data = {} 
week_groups_df = pd.DataFrame()

start_week = parse_date(start_date).isocalendar().week
end_week = parse_date(end_date).isocalendar().week

# Copy df into weeks
for repo_name in repo_names:
  weeks = pd.DataFrame(data_frames[repo_name])
  weeks['created_at'] = weeks['created_at'].dt.isocalendar().week

  weeks_data[repo_name] = weeks
  # TODO Figure out why it's dropping weeks 36 & 37...
#   week_groups_df[repo_name] = weeks.groupby(weeks['created_at'])['id'].count()

# week_groups_df.plot(kind='bar', figsize=(15,7), xlabel='PR Creation Week', ylabel='Count', title='PR Creations per Week by Service')

In [ ]:
# Compute total PRs by repo

pr_counts = {}

for repo_name in repo_names:
  pr_counts[repo_name] = data_frames[repo_name]['id'].count()

pr_counts_df = pd.DataFrame.from_dict(pr_counts, orient='index')

pr_counts_df.plot(kind='bar', title="Total PRs by Repo between week "+str(start_week)+" and "+str(end_week), xlabel="Repo Name", ylabel='Count', legend=None, grid=True, figsize=(15,7))

In [ ]:
# Compute total PRs by repo by team/non-team

pr_counts_with_team = {}

for repo_name in repo_names:
  pr_counts_with_team[repo_name] = {}
  on_team = data_frames[repo_name][data_frames[repo_name]['user.login'].isin(gh_account_list)]
  not_on_team = data_frames[repo_name][~data_frames[repo_name]['user.login'].isin(gh_account_list)]

  pr_counts_with_team[repo_name]['on_team'] = on_team['user.login'].count()
  pr_counts_with_team[repo_name]['not_on_team'] = not_on_team['user.login'].count()
  

pr_counts_with_team_df = pd.DataFrame.from_dict(pr_counts_with_team, orient='columns').T
print(pr_counts_with_team_df)

pr_counts_with_team_df.plot.bar(stacked=True, title="Total PRs by Repo between week "+str(start_week)+" and "+str(end_week), xlabel="Repo Name", ylabel='Count', grid=True, figsize=(15,7))